<a href="https://colab.research.google.com/github/slippy0/gpt-2/blob/finetuning/gpt2_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 297, done.
remote: Total 297 (delta 0), reused 0 (delta 0), pack-reused 297
Receiving objects: 100% (297/297), 4.39 MiB | 8.05 MiB/s, done.
Resolving deltas: 100% (162/162), done.
/content/gpt-2


In [0]:
%cd /content/gpt-2
!pip install -r requirements.txt
%cd /content

/content/gpt-2
/content


In [0]:
# Settings

# 117M or 345M
model_name = "117M"

In [0]:
!python gpt-2/download_model.py $model_name

Fetching checkpoint: 1.00kit [00:00, 899kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 57.8Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.09Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:07, 66.5Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 5.60Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 53.5Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 48.1Mit/s]                                                       


In [0]:
# UPLOAD .txt FILES TO /content FIRST
![ ! -d "./data" ] && mkdir data
%mv *.txt data

# Encode data
!PYTHONPATH=gpt-2/src gpt-2/encode.py \
    --model_name $model_name \
    data encoded_data.npz

Reading files
100% 1/1 [00:00<00:00,  5.09it/s]
Writing encoded_data.npz


In [0]:
# This runs forever. Stop manually when you're getting good samples
!PYTHONPATH=gpt-2/src gpt-2/train.py \
    --dataset encoded_data.npz \
    --model_name $model_name \
    --save_every 50 \
    --sample_every 50

2019-05-24 06:55:41.814643: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-05-24 06:55:41.814857: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2f5c680 executing computations on platform Host. Devices:
2019-05-24 06:55:41.814889: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-24 06:55:41.978256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-24 06:55:41.978763: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2f5bfa0 executing computations on platform CUDA. Devices:
2019-05-24 06:55:41.978794: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-24 06:55:41.979176: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

In [0]:
# generate_samples is hardcoded to this path, so copy our trained models there
!cp -r /content/checkpoint/run1/* /content/models/$model_name/

In [0]:
# Generate a random sample
!PYTHONPATH=gpt-2/src python gpt-2/src/generate_unconditional_samples.py \
    --model_name $model_name \
    --nsamples 1 \
    --top_p 0.9

In [0]:
# Generate samples with a prefix
!PYTHONPATH=gpt-2/src python gpt-2/src/interactive_conditional_samples.py \
    --model_name $model_name \
    --top_p 0.9

2019-05-24 06:58:38.931167: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-05-24 06:58:38.931427: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x306c520 executing computations on platform Host. Devices:
2019-05-24 06:58:38.931461: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-24 06:58:39.090166: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-24 06:58:39.090659: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x306be40 executing computations on platform CUDA. Devices:
2019-05-24 06:58:39.090688: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-24 06:58:39.091007: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

In [0]:
# Remove everything to start over
%rm -rf checkpoint
%rm -rf data
%rm -rf gpt-2